In [3]:
import pandas as pd
import numpy as np
import os

In [6]:
dfs = []
for filename in os.listdir('..\\data\\trainset_A'):
    filename = '..\\data\\trainset_A\\' + filename
    dfs.append(pd.read_csv(filename, sep=','))

In [11]:
def first(x):
    if x.first_valid_index() is None:
        return None
    else:
        return x[x.first_valid_index()]

def last(x):
    if x.last_valid_index() is None:
        return None
    else:
        return x[x.last_valid_index()]

In [12]:
dfsmale = []
dfsfemale = []
leng = len(dfs)
for i in range(leng):
    dfs[i] = dfs[i].pivot(columns = 'Parameter', values = 'Value').replace(-1,np.NaN)
    dfs[i]['RecordID'] = dfs[i]['RecordID'].fillna(dfs[i]['RecordID'].mean())
    if dfs[i]['Gender'].mean() == 0:
        dfsmale.append(dfs[i])
    else:
        dfsfemale.append(dfs[i])

In [36]:
dfmale = pd.concat(dfsmale,sort = False)
dffemale = pd.concat(dfsfemale, sort = False)

static = ['Age','Gender','Height','ICUType','RecordID']
male_static = dfmale[static].groupby(by = 'RecordID').agg(np.mean)
female_static = dffemale[static].groupby(by = 'RecordID').agg(np.mean)

static = static[:-1]

male_dynamic = dfmale.drop(static,axis = 1)
female_dynamic = dffemale.drop(static,axis = 1)

male_count = male_dynamic.groupby(by='RecordID').agg(pd.DataFrame.count)
male_count = male_count.add_suffix('_count')

male_mini = male_dynamic.groupby(by = 'RecordID').agg(np.min)
male_mini = male_mini.add_suffix('_mini')

male_maxi = male_dynamic.groupby(by = 'RecordID').agg(np.max)
male_maxi = male_maxi.add_suffix('_maxi')

male_first = male_dynamic.groupby(by = 'RecordID').agg(first)
male_first = male_first.add_suffix('_first')

male_last = male_dynamic.groupby(by = 'RecordID').agg(last)
male_last = male_last.add_suffix('_last')

male_median = male_dynamic.groupby(by = 'RecordID').agg(np.median)
male_median = male_median.add_suffix('_median')


female_count = female_dynamic.groupby(by='RecordID').agg(pd.DataFrame.count)
female_count = female_count.add_suffix('_count')

female_mini = female_dynamic.groupby(by = 'RecordID').agg(np.min)
female_mini = female_mini.add_suffix('_mini')

female_maxi = female_dynamic.groupby(by = 'RecordID').agg(np.max)
female_maxi = female_maxi.add_suffix('_maxi')

female_first = female_dynamic.groupby(by = 'RecordID').agg(first)
female_first = female_first.add_suffix('_first')

female_last = female_dynamic.groupby(by = 'RecordID').agg(last)
female_last = female_last.add_suffix('_last')

female_median = female_dynamic.groupby(by = 'RecordID').agg(np.median)
female_median = female_median.add_suffix('_median')

male = pd.concat([male_static, male_count, male_mini, male_maxi, male_first, male_last, male_median], axis = 1)
female = pd.concat([female_static, female_count, female_mini, female_maxi, female_first, female_last,female_median], axis = 1)

male = male.fillna(male.median())
female = female.fillna(female.median())

df = pd.concat([male, female],sort=False)
df = df.fillna(df.median())

df = pd.get_dummies(df, columns=['ICUType'], drop_first=True)

In [38]:
df.to_csv('extract.csv')

In [45]:
data = pd.read_csv('extract.csv')
outcome = pd.read_csv('outcome_A.txt')
merged = pd.merge(data, outcome,sort=False)
X = merged.drop(['SAPS-I','SOFA','Length_of_stay','Survival','In-hospital_death'], axis = 1)
y = merged['In-hospital_death']

In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [51]:
clf = RandomForestClassifier(n_estimators=500)
print(cross_val_score(clf, X, y,scoring='accuracy',cv = 10).mean())

0.8732531125194531


In [62]:
clf = SVC(kernel='poly')

In [57]:
from sklearn.model_selection import KFold
from sklearn import metrics

In [ ]:
kf = KFold(n_splits=10)
scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()
    score = min(tp/(tp+fn),tp/(tp+fp))
    print(score)